In [1]:
#TODO: update code for Lightning

# from google.colab import drive
# drive.mount('/content/drive')

# update folder name according to data 
# folder_training_data = "#14 Hi_ILY_nothnig wo facemesh"
# folder_training_data = DATA_PATH

# import sys
# sys.path.append('/content/drive/My Drive/Bootcamp/Final Project/exploring action recognition/models/'+folder_training_data)

# Sign Language Detection Project



In [4]:
# WARNING: updates of config may require restart of kernel if reload is unsuccessful
import config
# reload config without restarting the kernel
import importlib
importlib.reload(config)
from config import facemesh_included, actions, no_sequences, sequence_length, DATA_PATH, WEIGHTS_PATH 
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from model import create_model

import numpy as np
import os

2024-05-11 15:01:12.592509: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-11 15:01:13.362539: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-11 15:01:13.362703: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-05-11 15:01:13.441643: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-11 15:01:14.964589: W tensorflow/stream_executor/pla

In [5]:
# paths for keypoint files; update if needed
# X = np.load('/content/drive/MyDrive/Bootcamp/Final Project/exploring action recognition/models/#13 Hi_ILY_nothing/X.npy')
# y = np.load('/content/drive/MyDrive/Bootcamp/Final Project/exploring action recognition/models/#13 Hi_ILY_nothing/y.npy')
X = np.load(os.path.join(DATA_PATH,'X.npy'))
y = np.load(os.path.join(DATA_PATH,'y.npy'))

In [11]:
# training set size
test_size = 0.99
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
# TODO: replace with train, validate and test split

# # Assuming 'data' is your dataset loaded as an array of shape [num_samples, num_features]
# np.random.shuffle(data)  # Shuffling the data to randomize input

# num_samples = data.shape[0]
# train_end = int(num_samples * 0.7)
# val_end = int(num_samples * 0.9)

# train_data = data[:train_end]
# val_data = data[train_end:val_end]
# test_data = data[val_end:]

# 7. Build and Train LSTM Neural Network

In [5]:
log_dir = os.path.join('Logs')

In [8]:
# OPTIONAL model() arguments
# available model_types: "LSTM", "Conv2D"
# tested activation functions: "tanh", "relu"; OTHERS: see below 
# tested activations for last neural layer: "softmax", "sigmoid"; OTHERS: "linear", "tanh", "relu", "softplus", "softsign", "selu", "elu", "exponential"
# neural multiplicator: multiply neurons per layer by this amount (default = 1); the higher NM, the longer the computation
# output will change according if facemesh_included = True or False

# reLU = Rectified Linear Activation Function: returns 0 if it receives any negative input, but for any positive value x it returns that value back.  It helps the model to account for non-linearity.
# Sigmoid: Outputs a value between 0 and 1, making it suitable for binary classification or multilabel classification tasks where each label is predicted independently.
model_type, activation_function, activation, neural_multiplicator = "LSTM", "tanh", "softmax", 20
model = create_model(model_type, activation_function, activation, neural_multiplicator)

2024-05-11 15:02:11.356125: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-05-11 15:02:11.356723: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-11 15:02:11.356886: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-192-10-214): /proc/driver/nvidia/version does not exist
2024-05-11 15:02:11.360318: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-11 15:02:11.881833: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26

In [7]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  
    patience=10,          
    restore_best_weights=True  
)

# EARLYSTOPPING
# Assuming the goal of a training is to minimize the loss. With this, the metric to be monitored would be 'loss', 
# and mode would be 'min'. A model.fit() training loop will check at end of every epoch whether the loss is 
# no longer decreasing, considering the min_delta and patience if applicable. Once it's found no longer decreasing, 
# model.stop_training is marked True and the training terminates.
# The quantity to be monitored needs to be available in logs dict. To make it so, pass the loss or metrics at model.compile().

# ARGUMENTS
# monitor: The metric to monitor. Default is "val_loss".
# min_delta: Min change in the monitored metric to be considered an improvement.
#            Any change less than this is not considered. Default is 0.
# patience: Epochs to wait for improvement before stopping. Default is 0.
# verbose: Verbosity mode, 0 is silent, 1 displays messages. Default is 0.
# mode: Defines when to stop training. Options are "auto", "min", "max".
#       "min" stops when decrease in metric ceases.
#       "max" stops when increase in metric ceases.
#       "auto" infers direction from the monitored metric. Default is "auto".
# baseline: Baseline value for comparison. Stops if no improvement over this.
#           Default is None.
# restore_best_weights: Restores weights from the best epoch if True.
#                       Uses last training weights if False. Default is False.
# start_from_epoch: Waits this many epochs before monitoring. Allows for warm-up.
#                   Default is 0.


keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=10,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0,
)

tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=0,
    write_graph=True,
    write_images=False,
    write_steps_per_second=False,
    update_freq="epoch",
    profile_batch=0,
    embeddings_freq=0,
    embeddings_metadata=None,
)


# val_data = 
# hist = model.fit(X_train, y_train, epochs=2000, validation_data=val_data, callbacks=[early_stopping])
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])
# model.fit(X_train, y_train, epochs=2000, callbacks=[early_stopping])

Epoch 1/2000


5/5 [==============================] - 7s 471ms/step - loss: 2.3286 - categorical_accuracy: 0.2517
Epoch 2/2000
5/5 [==============================] - 2s 466ms/step - loss: 1.5635 - categorical_accuracy: 0.4762
Epoch 3/2000
5/5 [==============================] - 2s 444ms/step - loss: 1.1299 - categorical_accuracy: 0.5578
Epoch 4/2000
5/5 [==============================] - 3s 488ms/step - loss: 0.9364 - categorical_accuracy: 0.5646
Epoch 5/2000
5/5 [==============================] - 2s 446ms/step - loss: 0.9431 - categorical_accuracy: 0.6803
Epoch 6/2000
5/5 [==============================] - 3s 513ms/step - loss: 0.8821 - categorical_accuracy: 0.6190
Epoch 7/2000
5/5 [==============================] - 2s 446ms/step - loss: 0.8548 - categorical_accuracy: 0.6327
Epoch 8/2000
5/5 [==============================] - 3s 519ms/step - loss: 0.6844 - categorical_accuracy: 0.7347
Epoch 9/2000
5/5 [==============================] - 2s 444ms/step - loss: 0.5893 - categorical_accuracy: 0.7347
Epoch

KeyboardInterrupt: 

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            82688     
                                                                 
 lstm_1 (LSTM)               (None, 30, 1280)          6886400   
                                                                 
 lstm_2 (LSTM)               (None, 640)               4917760   
                                                                 
 dense (Dense)               (None, 640)               410240    
                                                                 
 dense_1 (Dense)             (None, 640)               410240    
                                                                 
 dense_2 (Dense)             (None, 7)                 4487      
                                                                 
Total params: 12,711,815
Trainable params: 12,711,815
No

## 8. Make predictions

In [9]:
res = model.predict(X_test)

2/2 [==============================] - 1s 168ms/step


In [10]:
actions[np.argmax(res[4])]

'ILoveYou'

In [11]:
actions[np.argmax(y_test[4])]

'Hi'

In [12]:
actions[np.argmax(res[3])]

'NoHands'

In [13]:
actions[np.argmax(y_test[3])]

'NoHands'

In [9]:
weights_name = "weights_wo_facemesh_LSTM_tanh_softmax_NM20_testsize0.3"
model.load_weights(os.path.join(WEIGHTS_PATH, weights_name + ".keras"))

In [14]:
len(X_test)

208

In [17]:
res = model.predict(X_test)
correct_count = 0
for i in range(len(res)):
  prediction = actions[np.argmax(res[i])]
  actual = actions[np.argmax(res[i])]
  if prediction == actual:
    correct_count += 1
  # print("prediction: ", prediction)
  # print("actual    : ", actual)
  # print("~"*30)
print("~"*30)
print("accuracy on x_test: ", correct_count/len(res)*100, "%"
f"correct_count {correct_count}")

7/7 [==============================] - 2s 293ms/step
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
accuracy on x_test:  100.0 %correct_count 208


## 9. Save weights

In [15]:
if facemesh_included == True:
    weights_name = f"weights_w_facemesh_{model_type}_{activation_function}_{activation}_NM{str(neural_multiplicator)}_testsize{test_size}"
elif facemesh_included == False:
    weights_name = f"weights_wo_facemesh_{model_type}_{activation_function}_{activation}_NM{str(neural_multiplicator)}_testsize{test_size}"
model.save(os.path.join(WEIGHTS_PATH, weights_name + ".keras"))

## 10. Evaluation using confusion matrix and accuracy

In [16]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [17]:
yhat = model.predict(X_train)

1/5 [=====>........................] - ETA: 0s

5/5 [==============================] - 1s 117ms/step


In [18]:
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [19]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[123,   0],
        [  0,  24]],

       [[126,   0],
        [  0,  21]],

       [[132,   0],
        [  0,  15]],

       [[124,   0],
        [  0,  23]],

       [[129,   0],
        [  0,  18]],

       [[123,   0],
        [  0,  24]],

       [[125,   0],
        [  0,  22]]])

In [20]:
accuracy_score(ytrue, yhat)

1.0

In [21]:
accuracy_score??

Signature: accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None)
Source:   
@validate_params(
    {
        "y_true": ["array-like", "sparse matrix"],
        "y_pred": ["array-like", "sparse matrix"],
        "normalize": ["boolean"],
        "sample_weight": ["array-like", None],
    },
    prefer_skip_nested_validation=True,
)
def accuracy_score(y_true, y_pred, *, normalize=True, sample_weight=None):
    """Accuracy classification score.

    In multilabel classification, this function computes subset accuracy:
    the set of labels predicted for a sample must *exactly* match the
    corresponding set of labels in y_true.

    Read more in the :ref:`User Guide <accuracy_score>`.

    Parameters
    ----------
    y_true : 1d array-like, or label indicator array / sparse matrix
        Ground truth (correct) labels.

    y_pred : 1d array-like, or label indicator array / sparse matrix
        Predicted labels, as returned by a classifier.

    normalize : bool, defaul